In [ ]:
#!/opt/conda/envs/dsenv/bin/python

import sys
import os
from glob import glob
import logging

sys.path.append('.')
from model import fields

fields.remove('label')
#
# Init the logger
#
logging.basicConfig(level=logging.DEBUG)
logging.info("CURRENT_DIR {}".format(os.getcwd()))
logging.info("SCRIPT CALLED AS {}".format(sys.argv[0]))
logging.info("ARGS {}".format(sys.argv[1:]))

#
# import the filter
#
filter_cond_files = glob('filter_cond*.py')
logging.info(f"FILTERS {filter_cond_files}")

if len(filter_cond_files) != 1:
    logging.critical("Must supply exactly one filter")
    sys.exit(1)

exec(open(filter_cond_files[0]).read())

#
# dataset fields
#
#fields = """doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,
#num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource""".replace("\n",'').split(",")

#
# Optional argument
# If +field option is given, output the id (always first record) and the given field
# if -field is given, output all but the given field
#

if len(sys.argv) == 1:
  #by default print all fields
  outfields = fields
else:
  op, field = sys.argv[1][0], sys.argv[1][1:]
  logging.info(f"OP {op}")
  logging.info(f"FIELD {field}")

  if not op in "+-" or not field in fields:
    logging.critical("The optional argument must start with + or - followed by a valid field")
    sys.exit(1)
  elif op == '+':
    outfields = [fields[0], field]
  else:
    outfields = list(fields) # like deepcopy, but on the first level only!
    outfields.remove(field)



for line in sys.stdin:
    # skip header
    if line.startswith(fields[0]):
        continue

    #unpack into a tuple/dict
    values = line.rstrip().split('\t')
    ad_record = dict(zip(fields, values)) #Hotel(values)

    #apply filter conditions
    if filter_cond(ad_record):
        output = "\t".join([ad_record[x] for x in outfields])
        print(output)


In [17]:
def filter_cond(line_dict):
    """Filter function
    Takes a dict with field names as argument
    Returns True if conditions are satisfied
    """
    cond_match = False if line_dict["if1"] == '' else ((int(line_dict["if1"]) > 20) & (int(line_dict["if1"]) < 40)) 
    return True if cond_match else False

In [ ]:
projects/1/filter.sh projects/1/filter.py,projects/1/filter_cond.py,projects/1/model.py /datasets/criteo_valid_large_features filtered.csv filter.py




In [1]:
from model import fields

In [2]:
fields.remove('label')

In [4]:
path = '/home/users/ValentinMSU17/ozon-masters-bigdata/filter_test'

In [19]:
outfields = fields

with open(path, 'r') as f:
    line = f.readline()
    while line:
        #print(line)
        values = line.rstrip().split('\t')
        ad_record = dict(zip(fields, values)) #Hotel(values)

        #apply filter conditions
        if filter_cond(ad_record):
            output = "\t".join([ad_record[x] for x in outfields])
            print(output)
        line = f.readline()

In [14]:
int(ad_record["if1"]) > 20

ValueError: invalid literal for int() with base 10: ''

In [15]:
ad_record["if1"]

''

In [9]:
ad_record = dict(zip(fields, values))

In [10]:
ad_record

{'id': '70809833',
 'if1': '40',
 'if2': '19',
 'if3': '',
 'if4': '30',
 'if5': '8',
 'if6': '0',
 'if7': '0',
 'if8': '41',
 'if9': '2',
 'if10': '0',
 'if11': '1',
 'if12': '7384',
 'if13': '',
 'cf1': 'ad98e872',
 'cf2': 'fb1e95da',
 'cf3': 'dff8c25a',
 'cf4': 'baa22edd',
 'cf5': '38e8e49d',
 'cf6': '6fcd6dcb',
 'cf7': 'c12b72eb',
 'cf8': '3f8464ea',
 'cf9': '25dd8f9a',
 'cf10': '62da11e3',
 'cf11': '14874876',
 'cf12': '2dc365bb',
 'cf13': 'a77a4a56',
 'cf14': 'be4ee537',
 'cf15': 'c375920d',
 'cf16': '4cdc3efa',
 'cf17': 'd20856aa',
 'cf18': 'a1eb1511',
 'cf19': 'cc7a7d35',
 'cf20': '156cbe87',
 'cf21': '96fbe197',
 'cf22': '15562d5d',
 'cf23': 'd3df7183',
 'cf24': 'e59d5dec',
 'cf25': '321935cd',
 'cf26': 'e1be5ef2',
 'day_number': 'day_7'}

In [ ]:
if filter_cond(ad_record):
        output = "\t".join([ad_record[x] for x in outfields])
        print(output)

In [11]:
filter_cond(ad_record)

False

In [16]:
values

['70809834',
 '',
 '10',
 '',
 '2',
 '3',
 '0',
 '1',
 '-1',
 '1',
 '0',
 '2',
 '5774',
 '',
 '65839842',
 '38ccbc8f',
 '330ce00c',
 'df02c0aa',
 '729e35ab',
 '6fcd6dcb',
 'fd13d94f',
 '038e402d',
 '2e4e821f',
 '7bab0c82',
 '4a17f680',
 '6d85a359',
 'f4fc3c5e',
 '',
 '37de4a42',
 '',
 '',
 'a1eb1511',
 '9512c20b',
 'd7003867',
 'e0346dff',
 '5efb86a8',
 '',
 'f8dfa135',
 'ff654802',
 'b757e957',
 'day_7']

In [ ]:
projects/1/filter_predict.sh projects/1/filter.py,projects/1/predict.py,projects/1/filter_cond.py,1.joblib,projects/1/model.py /datasets/criteo_valid_large_features pred_with_filter filter.py predict.py


In [ ]:
hadoop fs -cat pred_with_filter/part-* > pred_with_filter_cat

projects/1/scorer_local.py /home/users/datasets/criteo_valid_large_filtered_labels  'http://head1:50070/webhdfs/v1/user/ValentinMSU17/pred_with_filter_cat?op=OPEN'
